In [23]:
import pandas as pd
import numpy as np 
import glob 
import warnings 
from collections import Counter
warnings.filterwarnings("ignore")
import plotly.express as px 
import seaborn as sns 
import matplotlib.pyplot as plt 
import lightgbm as lgbm 
from sklearn.model_selection import StratifiedKFold, train_test_split
import math
import os 
import random
import torch 
import pyarrow as pa
import pyarrow.parquet as pq
import torch.nn as nn
#from transformers import AdamW
from torch.utils.data import Dataset , DataLoader
from colorama import Fore , Style
r__=Fore.RED
g__=Fore.GREEN
st__=Style.RESET_ALL

In [24]:
train = pd.read_csv("optiver_raw_data/train.csv")
# get a look at train data.
train.head() 

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [25]:
#Split training data into 80% train + 20% test for stocks 0-5

train = pd.read_csv("optiver_raw_data/train.csv")

time_id_of_first_80_percent = train.iloc[18384,1]


# get a look at train data for the stocks 0-5.
train = train[train["stock_id"].isin([0,1,2,3,4,5])]
train


,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747
...,...,...,...
22975,5,32751,0.003966
22976,5,32753,0.001542
22977,5,32758,0.003377
22978,5,32763,0.006443


In [26]:
stock_0 = train[train["stock_id"]==0]
df_train = pd.DataFrame()
df_test = pd.DataFrame()
for i in range(6):
    stock_i = train[train["stock_id"]==i]
    split_list_i = train_test_split(stock_i, train_size = 0.8, test_size=0.2, random_state=1) 
    df_train = pd.concat([df_train, split_list_i[0]])
    df_test = pd.concat([df_test, split_list_i[1]])

In [27]:
df_train.to_csv('target_train_0to5.csv')
df_test.to_csv('target_test_0to5.csv')

In [28]:
#Split training data into 80% train + 20% test for stocks 0-5
book_train_stock_id_0 = pd.read_parquet("optiver_raw_data/book_train.parquet/stock_id=0")
book_train_stock_id_0



stock_0_train = book_train_stock_id_0[book_train_stock_id_0['time_id'] <= time_id_of_first_80_percent]
stock_0_train.to_csv('stock_train/stock_0_train.csv')

stock_0_test = book_train_stock_id_0[book_train_stock_id_0['time_id'] > time_id_of_first_80_percent]
stock_0_test.to_csv('stock_test/stock_0_test.csv')



In [29]:
train_test_split(book_train_stock_id_0, train_size = 0.8, test_size=0.2, random_state=1)

[        time_id  seconds_in_bucket  bid_price1  ask_price1  bid_price2  \
 74663      2468                488    0.993515    0.994133    0.993468   
 848223    30210                 30    0.999725    1.000375    0.999675   
 68189      2259                285    0.997960    0.998348    0.997863   
 813819    28745                343    1.001068    1.001912    1.001018   
 23963       904                 56    0.999541    1.000217    0.999155   
 ...         ...                ...         ...         ...         ...   
 491263    16919                331    1.000645    1.001409    1.000454   
 791624    27828                423    1.002388    1.003302    1.001981   
 470924    16240                534    0.997424    0.999009    0.997375   
 491755    16943                404    1.013062    1.015816    1.012777   
 128037     4361                314    0.999886    1.000114    0.999840   
 
         ask_price2  bid_size1  ask_size1  bid_size2  ask_size2  
 74663     0.994275         45  